In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stats

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import KBinsDiscretizer
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score

In [2]:
from sklearn.datasets import make_regression

In [3]:
from sklearn.datasets import load_diabetes

In [4]:
X,y = load_diabetes(return_X_y=True)

In [5]:
print(X.shape)
print(y.shape)

(442, 10)
(442,)


### **TRAIN TEST SPLIT**

In [6]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)

In [7]:
reg = LinearRegression()
reg.fit(X_train,y_train)

LinearRegression()

In [8]:
reg.coef_

array([  -9.15865318, -205.45432163,  516.69374454,  340.61999905,
       -895.5520019 ,  561.22067904,  153.89310954,  126.73139688,
        861.12700152,   52.42112238])

In [9]:
reg.intercept_

151.88331005254167

In [10]:
y_pred = reg.predict(X_test)
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)

0.4399338661568968

In [11]:
np.mean(cross_val_score(reg,X_train,y_train,scoring='r2',cv=10))

0.484764012049682

### **CREATING A CLASS**

In [12]:
import random

In [13]:
class MBGDRegressor:

    def __init__(self,batch_size,learning_rate=0.01,epochs=100):

        self.coef_ = None
        self.intercept_ = None
        self.lr = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size

    def fit(self,X_train,y_train):
        # init your coefs
        self.intercept_ = 0
        self.coef_ = np.ones(X_train.shape[1])

        for i in range(self.epochs):

            for j in range(int(X_train.shape[0]/self.batch_size)):

                idx = random.sample(range(X_train.shape[0]),self.batch_size)

                y_hat = np.dot(X_train[idx],self.coef_) + self.intercept_
                #print("Shape of y_hat",y_hat.shape)
                intercept_der = -2 * np.mean(y_train[idx] - y_hat)
                self.intercept_ = self.intercept_ - (self.lr * intercept_der)

                coef_der = -2 * np.dot((y_train[idx] - y_hat),X_train[idx])
                self.coef_ = self.coef_ - (self.lr * coef_der)

        print(self.intercept_,self.coef_)

    def predict(self,X_test):
        return np.dot(X_test,self.coef_) + self.intercept_

In [14]:
#import random
#random.sample(range(1,100),10)

In [15]:
mbr = MBGDRegressor(batch_size=int(X_train.shape[0]/50),learning_rate=0.01,epochs=100)

In [16]:
mbr.fit(X_train,y_train)

157.73371009218621 [  28.68263604 -139.15926703  455.82194145  295.30943253  -25.32004201
  -94.5481409  -196.36956687  113.7892492   414.08169545  110.25433767]


In [17]:
X_test.shape

(89, 10)

In [18]:
y_pred = mbr.predict(X_test)

In [19]:
r2_score(y_test,y_pred)

0.44886822190096054

In [20]:
X_train.shape

(353, 10)

In [22]:
int(350/50)

7

the number of rows are 350 and the batch size is determined to be 10 so we got 35 batches

In [23]:
from sklearn.linear_model import SGDRegressor

In [25]:
sgd = SGDRegressor(learning_rate='constant',eta0=0.1)

In [26]:
batch_size= 35

for i in  range (100):

    idx = random.sample(range(X_train.shape[0]),batch_size)
    sgd.partial_fit(X_train[idx],y_train[idx])

In [27]:
sgd.coef_

array([  45.47632482,  -58.09913742,  331.78904461,  229.41762897,
         27.49896923,  -22.74988893, -176.47601863,  138.38796379,
        324.09495685,  125.03364138])

In [28]:
sgd.intercept_

array([176.16716135])

In [30]:
y_pred1 = sgd.predict(X_test)

In [31]:
r2_score(y_test,y_pred)

0.44886822190096054